In [19]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ readr   2.1.2
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract()        masks magrittr::extract()
✖ dplyr::filter()         masks stats::filter()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()
✖ purrr::set_names()      masks magrittr::set_names()


In [20]:
library(magrittr) # better handling of pipes
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(stringr) # to hand strings

In [21]:
# install.packages("rvest")
# remotes::install_github("dmi3kno/polite")("politer")
# install.packages("xml2")
library(rvest) # rvest makes scraping easier
library(polite) # polite is the "polite" version of rvest
library(xml2) # makes it easier to work with HTML and XML from R

In [16]:
auckland_url <- "https://www.weatherwatch.co.nz/forecasts/Auckland" 
wellington_url <- "https://www.weatherwatch.co.nz/forecasts/Wellington"
hamilton_url <- "https://www.weatherwatch.co.nz/forecasts/Hamilton"
tauranga_url <- "https://www.weatherwatch.co.nz/forecasts/tauranga"
christchurch_url <- "https://www.weatherwatch.co.nz/forecasts/christchurch"
dunedin_url <- "https://www.weatherwatch.co.nz/forecasts/dunedin"

In [8]:
# Read the HTML page by passing the url 
get_location_html <- function(location_url) {
    location_html <- read_html(location_url) # Use read_html() function to get the HTML using the location url
    return(location_html) # Return the result of reading the HTML
}
# Check the result of reading Auckland html
auckland_html <- get_location_html(auckland_url)
auckland_html

{html_document}
<html>
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body>\n<div id="__next">\n<div class="jsx-3479055027 browser"><div class ...

In [10]:
# Getting the date for the particular location using the selector
get_dates <- function(location_html) {
    dates <- location_html %>% html_nodes(".date") %>% html_text() # Using selector date to get the date
    index <- 1:18 # Since the result being passed doubled or duplicate data, we only take one 
    date <- dates[index%%2==0]
    return(date)
}
# Check if it is returning the right date
auckland_date <- get_dates(auckland_html)
auckland_date

[1] "Wed 19 Oct" "Thu 20 Oct" "Fri 21 Oct" "Sat 22 Oct" "Sun 23 Oct"
[6] "Mon 24 Oct" "Tue 25 Oct" "Wed 26 Oct" "Thu 27 Oct"

In [11]:
# Getting the maximum temperature for the particular location
get_max_temp <- function(location_html) {
    max_temp <- location_html %>% html_nodes(".high") %>% html_text() # Using the selector .high to select the temperature
    return(max_temp)
}
# Check if it is returning the right temperature
auckland_max_temp <- get_max_temp(auckland_html)
auckland_max_temp

[1] "17°Day" "17°Day" "17°Day" "18°Day" "19°Day" "19°Day" "19°Day" "19°Day"
[9] "19°Day"

In [12]:
# Getting the minimum temperature for the particular location
get_min_temp <- function(location_html) {
    min_temp <- location_html %>% html_nodes(".low") %>% html_text() # Using the selector .low to select the temperature
    return(min_temp)
}
# Check if it is returning the right temperature
auckland_min_temp <- get_min_temp(auckland_html)
auckland_min_temp

[1] "12°Night" "10°Night" "10°Night" "11°Night" "13°Night" "12°Night" "13°Night"
[8] "13°Night" "14°Night"

In [13]:
# Getting the weather from the particular location
get_weathers <- function(location_html) {
    weathers <- location_html %>% html_nodes(".larger") %>% html_text() # Using the selector called .larger to get the weather condition for the particular location
    index <- 1:18
    weather <- weathers[index%%2==0]
    return(weather)
}
# Check if it is returning the right data
auckland_weather <- get_weathers(auckland_html)
auckland_weather

[1] "Showers. Breezy to brisk West to Sou'West winds."
[2] "Showers. Fairly breezy South to Sou'West winds." 
[3] "Mostly sunny. Breezy East to Nor'East winds."    
[4] "Mainly sunny. Fairly breezy Sou'West winds."     
[5] "Partly cloudy. Fairly breezy Sou'West winds."    
[6] "Partly cloudy. Fairly breezy Sou'West winds."    
[7] "Mostly cloudy. Breezy West to Sou'West winds."   
[8] "Morning showers. Fairly breezy Nor'West winds."  
[9] "Light rain. Breezy Northerly winds."

In [14]:
# Getting the wind condition from the particular location
get_wind <- function(location_html) {
    wind <- location_html %>% html_nodes(".wind .small") %>% html_text() # Using the selector called .wind and .small to get the wind condition for the particular location
    return(wind)
}
# Check if it is returning the right data
auckland_wind <- get_wind(auckland_html)
auckland_wind

[1] "31km/h" "23km/h" "14km/h" "21km/h" "26km/h" "22km/h" "19km/h" "20km/h"
[9] "19km/h"

In [15]:
# Getting the location data for the data frame using the html 
get_location <- function(location_html) {
    location <- location_html %>% html_nodes("h2") %>% html_text() # Using the selector called h2 to get the location of the city
    return(location)
}
# Check if it is returning the right data
auckland_location <- get_location(auckland_html)
auckland_location

[1] "Weather Forecast for Auckland"

##### Now we have all the individual process to getting the data, we want to combined everything so we only need to call one function to get the data frame. This function will make us getting the data frame faster and more efficient for each location

In [ ]:
# We combined all the function to make the process of generating data frame easier
get_dataframe <- function(location_url) {
     location_html <- get_location_html(location_url)
     location <- get_location(location_html)
     date <- get_dates(location_html)
     max_temp <- get_max_temp(location_html)
     min_temp <- get_min_temp(location_html)
     weather <- get_weathers(location_html)
     wind_speed <- get_wind(location_html)
     
     weather_df <- tibble(Location=location, Date=date, Weather=weather, Min_temp=min_temp, Max_temp=max_temp, Wind_Speed=wind_speed)
     return(weather_df)
 }
auckland_df <- get_dataframe(auckland_url)
auckland_df

Location,Date,Weather,Min_temp,Max_temp,Wind_Speed
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Weather Forecast for Auckland,Wed 19 Oct,Showers. Breezy to brisk West to Sou'West winds.,12°Night,17°Day,31km/h
Weather Forecast for Auckland,Thu 20 Oct,Showers. Fairly breezy South to Sou'West winds.,10°Night,18°Day,23km/h
Weather Forecast for Auckland,Fri 21 Oct,Mainly sunny. Breezy Easterly winds.,10°Night,17°Day,14km/h
Weather Forecast for Auckland,Sat 22 Oct,Mostly sunny. Fairly breezy Sou'West winds.,12°Night,19°Day,21km/h
Weather Forecast for Auckland,Sun 23 Oct,Partly cloudy. Fairly breezy Sou'West winds.,13°Night,19°Day,26km/h
Weather Forecast for Auckland,Mon 24 Oct,Partly cloudy. Fairly breezy Sou'West winds.,13°Night,19°Day,22km/h
Weather Forecast for Auckland,Tue 25 Oct,Morning showers. Breezy West to Sou'West winds.,13°Night,19°Day,19km/h
Weather Forecast for Auckland,Wed 26 Oct,Mostly cloudy. Fairly breezy North to Nor'West winds.,14°Night,19°Day,20km/h
Weather Forecast for Auckland,Thu 27 Oct,Light rain. Breezy Northerly winds.,14°Night,20°Day,19km/h


In [23]:
# Now we want to see the dataframe for wellington
wellington_df <- get_dataframe(wellington_url)
wellington_df

Location,Date,Weather,Min_temp,Max_temp,Wind_Speed
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Weather Forecast for Wellington,Wed 19 Oct,Mostly sunny. Breezy to brisk North to Nor'West winds.,9°Night,16°Day,32km/h
Weather Forecast for Wellington,Thu 20 Oct,Windy and partly cloudy. Strong and blustery Southerly winds.,8°Night,11°Day,46km/h
Weather Forecast for Wellington,Fri 21 Oct,Partly cloudy. Breezy Sou'East winds.,9°Night,14°Day,18km/h
Weather Forecast for Wellington,Sat 22 Oct,Generally clear and windy. Brisk Northerly winds.,12°Night,16°Day,42km/h
Weather Forecast for Wellington,Sun 23 Oct,Partly cloudy. Fairly breezy Northerly winds.,11°Night,16°Day,25km/h
Weather Forecast for Wellington,Mon 24 Oct,Partly cloudy. Fairly breezy Nor'East winds.,11°Night,15°Day,23km/h
Weather Forecast for Wellington,Tue 25 Oct,Windy and partly cloudy. Breezy to brisk Northerly winds.,10°Night,15°Day,32km/h
Weather Forecast for Wellington,Wed 26 Oct,Partly cloudy. Breezy to brisk Northerly winds.,12°Night,15°Day,32km/h
Weather Forecast for Wellington,Thu 27 Oct,Showers. Breezy to brisk North to Nor'West winds.,11°Night,15°Day,32km/h


In [28]:
# Now we want to see the dataframe for hamilton
hamilton_df <- get_dataframe(hamilton_url)
hamilton_df

Location,Date,Weather,Min_temp,Max_temp,Wind_Speed
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Weather Forecast for Hamilton,Wed 19 Oct,Light rain. Breezy to brisk West to Sou'West winds.,10°Night,18°Day,30km/h
Weather Forecast for Hamilton,Thu 20 Oct,Showers. Breezy South to Sou'East winds.,4°Night,19°Day,15km/h
Weather Forecast for Hamilton,Fri 21 Oct,Mostly sunny. Light South to Sou'East winds.,3°Night,19°Day,11km/h
Weather Forecast for Hamilton,Sat 22 Oct,Mainly sunny. Breezy Sou'West winds.,7°Night,20°Day,19km/h
Weather Forecast for Hamilton,Sun 23 Oct,Partly cloudy. Fairly breezy Sou'West winds.,8°Night,21°Day,25km/h
Weather Forecast for Hamilton,Mon 24 Oct,Partly cloudy. Breezy South to Sou'West winds.,10°Night,21°Day,18km/h
Weather Forecast for Hamilton,Tue 25 Oct,Mostly cloudy. Breezy South to Sou'West winds.,9°Night,21°Day,19km/h
Weather Forecast for Hamilton,Wed 26 Oct,Morning showers. Breezy Northerly winds.,10°Night,21°Day,17km/h
Weather Forecast for Hamilton,Thu 27 Oct,Light rain. Breezy North to Nor'East winds.,11°Night,21°Day,19km/h


In [25]:
# Now we want to see the dataframe for tauranga
tauranga_df <- get_dataframe(tauranga_url)
tauranga_df

Location,Date,Weather,Min_temp,Max_temp,Wind_Speed
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Weather Forecast for Tauranga,Wed 19 Oct,Showers. Fairly breezy West to Sou'West winds.,11°Night,17°Day,25km/h
Weather Forecast for Tauranga,Thu 20 Oct,Afternoon showers. Breezy South to Sou'East winds.,6°Night,17°Day,15km/h
Weather Forecast for Tauranga,Fri 21 Oct,Mostly sunny. Light East to Nor'East winds.,7°Night,16°Day,12km/h
Weather Forecast for Tauranga,Sat 22 Oct,Mainly sunny. Breezy Westerly winds.,10°Night,20°Day,14km/h
Weather Forecast for Tauranga,Sun 23 Oct,Mainly sunny. Breezy West to Sou'West winds.,11°Night,21°Day,18km/h
Weather Forecast for Tauranga,Mon 24 Oct,Partly cloudy. Light West to Nor'West winds.,12°Night,20°Day,12km/h
Weather Forecast for Tauranga,Tue 25 Oct,Mostly cloudy. Light Nor'West winds.,11°Night,19°Day,13km/h
Weather Forecast for Tauranga,Wed 26 Oct,Partly cloudy. Light North to Nor'West winds.,12°Night,19°Day,12km/h
Weather Forecast for Tauranga,Thu 27 Oct,Light rain. Breezy Northerly winds.,13°Night,18°Day,14km/h


In [26]:
# Now we want to see the dataframe for christchurch
christchurch_df <- get_dataframe(christchurch_url)
christchurch_df

Location,Date,Weather,Min_temp,Max_temp,Wind_Speed
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Weather Forecast for Christchurch,Wed 19 Oct,Mostly sunny. Breezy North to Nor'East winds.,7°Night,16°Day,17km/h
Weather Forecast for Christchurch,Thu 20 Oct,Partly cloudy. Breezy to brisk South to Sou'West winds.,4°Night,11°Day,30km/h
Weather Forecast for Christchurch,Fri 21 Oct,Mainly sunny. Breezy Nor'East winds.,6°Night,14°Day,18km/h
Weather Forecast for Christchurch,Sat 22 Oct,Mostly sunny. Light Northerly winds.,9°Night,21°Day,13km/h
Weather Forecast for Christchurch,Sun 23 Oct,Mostly cloudy. Light Southerly winds.,9°Night,17°Day,13km/h
Weather Forecast for Christchurch,Mon 24 Oct,Morning showers. Breezy Easterly winds.,8°Night,15°Day,14km/h
Weather Forecast for Christchurch,Tue 25 Oct,Morning clouds followed by afternoon sun. Breezy Easterly winds.,9°Night,17°Day,14km/h
Weather Forecast for Christchurch,Wed 26 Oct,Partly cloudy. Breezy Nor'East winds.,10°Night,16°Day,14km/h
Weather Forecast for Christchurch,Thu 27 Oct,Afternoon showers. Breezy North to Nor'East winds.,10°Night,18°Day,14km/h


In [27]:
# Now we want to see the dataframe for dunedin
dunedin_df <- get_dataframe(dunedin_url)
dunedin_df

Location,Date,Weather,Min_temp,Max_temp,Wind_Speed
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Weather Forecast for Dunedin,Wed 19 Oct,"Windy, with light rain in the afternoon. Brisk West to Nor'West winds.",5°Night,14°Day,40km/h
Weather Forecast for Dunedin,Thu 20 Oct,Morning showers. Breezy to brisk Sou'West winds.,2°Night,9°Day,29km/h
Weather Forecast for Dunedin,Fri 21 Oct,Mainly sunny. Breezy North to Nor'East winds.,6°Night,14°Day,17km/h
Weather Forecast for Dunedin,Sat 22 Oct,Partly cloudy. Light North to Nor'East winds.,9°Night,17°Day,10km/h
Weather Forecast for Dunedin,Sun 23 Oct,Light rain. Breezy Southerly winds.,6°Night,11°Day,14km/h
Weather Forecast for Dunedin,Mon 24 Oct,Showers. Light Easterly winds.,7°Night,11°Day,13km/h
Weather Forecast for Dunedin,Tue 25 Oct,Morning showers. Breezy Sou'West winds.,7°Night,13°Day,16km/h
Weather Forecast for Dunedin,Wed 26 Oct,Afternoon showers. Breezy Nor'West winds.,8°Night,14°Day,14km/h
Weather Forecast for Dunedin,Thu 27 Oct,Showers. Breezy East to Nor'East winds.,8°Night,13°Day,18km/h
